In [285]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import time
import os
import gc
import nltk

np.random.seed(42)

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [28]:
df = pd.read_csv('products_sentiment_train.tsv', delimiter='\t', header=None, names=['Reviews', 'Target'])
df.head(5)

,Reviews,Target
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [29]:
df.isnull().values.any()

False

In [30]:
print ('Number of docs in session: {}'.format(len(df)))
np.bincount(df['Target'])

Number of docs in session: 2000


array([ 726, 1274])

In [31]:
# оп, у нас несбалансированная по таргету выборка. Сделаем stratify по таргету в train_test_split как минимум.
# Будем пробовать SVM
# Еще добавим наивный байесовский классификатор ну и конечно RF :)
# Где сможем, попробуем class_weight='balanced', штрафующий меньше за ошибки в минорном классе.

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [360]:
X_train = df['Reviews']
y = df['Target']

In [132]:
X_test = pd.read_csv('products_sentiment_test.tsv', sep='\t', usecols=[1])
X_test.head(5)

,text
0,"so , why the small digital elph , rather than ..."
1,3/4 way through the first disk we played on it...
2,better for the zen micro is outlook compatibil...
3,6 . play gameboy color games on it with goboy .
4,"likewise , i 've heard norton 2004 professiona..."


In [214]:
cv = CountVectorizer(stop_words='english')
X = cv.fit_transform(X_train)

In [36]:
LR = LogisticRegression(random_state=42, n_jobs=-1)

In [37]:
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

In [17]:
np.mean(cross_val_score(LR, X, y, scoring = 'accuracy', cv=rskf))

0.7668011681323008

In [18]:
svm = SVC(kernel='linear', random_state=42)
np.mean(cross_val_score(svm, X, y, scoring = 'accuracy', cv=rskf))

0.7491495034343965

In [42]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB

In [463]:
MNB = MultinomialNB()
np.mean(cross_val_score(MNB, X, y, scoring = 'accuracy', cv=rskf))

0.7794979449871562

In [229]:
CNB = ComplementNB()
np.mean(cross_val_score(CNB, X, y, scoring = 'accuracy', cv=rskf))

0.7775475652972831

In [233]:
knc = KNeighborsClassifier(n_neighbors=4, n_jobs=-1)
np.mean(cross_val_score(knc, X, y, scoring = 'accuracy', cv=5))

0.6624853905336908

In [192]:
knc = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
np.mean(cross_val_score(knc, X, y, scoring = 'accuracy', cv=5))

0.731496824980156

In [378]:
sgd = SGDClassifier(random_state=42, n_jobs=-1, loss='hinge', penalty='l2')
np.mean(cross_val_score(sgd, X, y, scoring = 'accuracy', cv=rskf))

0.7421381074256714

In [287]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(reviews)

In [41]:
MNB = MultinomialNB()
np.mean(cross_val_score(MNB, X, y, scoring = 'accuracy', cv=rskf))

NameError: name 'MultinomialNB' is not defined

In [ ]:
#наивный байесовский ожидаемо хуже с TFIDF

In [236]:
svm = SVC(kernel='linear', random_state=42)
np.mean(cross_val_score(svm, X, y, scoring = 'accuracy', cv=rskf))

0.7688974299839373

In [288]:
np.std(cross_val_score(svm, X, y, scoring = 'accuracy', cv=rskf))

0.020652658703663684

In [237]:
np.mean(cross_val_score(LR, X, y, scoring = 'accuracy', cv=rskf))

0.7666507946924668

In [39]:
RF = RandomForestClassifier()
np.mean(cross_val_score(RF, X, y, scoring = 'accuracy', cv=5))

0.7190305533159582

In [198]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [199]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
# rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = RF, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 14.1min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [201]:
rf_random.best_params_, rf_random.best_score_

({'n_estimators': 400,
  'min_samples_split': 2,
  'min_samples_leaf': 1,
  'max_features': 'sqrt',
  'max_depth': None,
  'bootstrap': False},
 0.7505)

In [ ]:
#переберем гиперпараметры для svm и посмотрим, что сможем выжать из NB с countvectorizer-ом

In [194]:
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.8, stratify=y)

In [ ]:
MultinomialNB

In [63]:
PipeNB = Pipeline([
        ('vectorizer', CountVectorizer()),
        ('MNB', MultinomialNB())
    ])

In [65]:
param_grid = [{'vectorizer__analyzer':['word', 'char_wb'],

               'vectorizer__stop_words':['english', None],
               'MNB__fit_prior': [True, False]
              }]
GSMNB = GridSearchCV(PipeNB, param_grid = param_grid, cv=5)
GSMNB.fit(X_train, y)
GSMNB.best_params_, GS.best_score_

({'MNB__fit_prior': True,
  'vectorizer__analyzer': 'word',
  'vectorizer__stop_words': None},
 0.779)

In [21]:
PipeLR = Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('lr', LogisticRegression(random_state=42, n_jobs=-1))
    ])

In [19]:
from nltk.corpus import stopwords

In [20]:
from sklearn.model_selection import RandomizedSearchCV

In [22]:
%%time
param_grid = [{'vectorizer__min_df':[0.7, 0.8, 0.9, 1, 2],
               'vectorizer__ngram_range':[(2, 5), (2, 6), (2, 7), (2, 8)],
               'vectorizer__analyzer':['char','char_wb'],
               'vectorizer__stop_words':['english', None],
               'lr__C' : [0.01, 0.05, 0.1, 0.5, 1, 2]
              }]
GS = GridSearchCV(PipeLR, param_grid = param_grid, cv=5)
GS.fit(X_train, y)
GS.best_params_, GS.best_score_

CPU times: user 41min 51s, sys: 9.84 s, total: 42min 1s
Wall time: 29min 44s


In [23]:
GS.best_params_, GS.best_score_

({'lr__C': 2,
  'vectorizer__analyzer': 'char_wb',
  'vectorizer__min_df': 2,
  'vectorizer__ngram_range': (2, 6),
  'vectorizer__stop_words': 'english'},
 0.789)

In [147]:
PipeLR = Pipeline([
        ('vectorizer', TfidfVectorizer(analyzer='char_wb', min_df = 2, ngram_range=(2,6), stop_words='enlish')),
        ('LR', LogisticRegression(C = 2, random_state=42, n_jobs=-1))
    ])

In [168]:
LR = LogisticRegression(C = 2, random_state=42, n_jobs=-1)

In [169]:
vectorizer = TfidfVectorizer(analyzer='char_wb', min_df = 2, ngram_range=(2,6), stop_words='enlish')

In [170]:
X = vectorizer.fit_transform(X_train)

In [171]:
X

<2000x19919 sparse matrix of type '<class 'numpy.float64'>'
	with 469683 stored elements in Compressed Sparse Row format>

In [172]:
X_te = vectorizer.transform(X_test.text)

In [173]:
X_te

<500x19919 sparse matrix of type '<class 'numpy.float64'>'
	with 118393 stored elements in Compressed Sparse Row format>

In [174]:
LR.fit(X, y)

LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l2', random_state=42,
                   solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [175]:
predict_lr = LR.predict(X_te)

In [184]:
submission = pd.read_csv('products_sentiment_sample_submission.csv')
submission['y'] = predict_lr
submission.to_csv('LR_predict.csv', index = None)

submission.tail()

,Id,y
495,495,1
496,496,1
497,497,0
498,498,0
499,499,1


In [42]:
PipeSVM = Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('svm', SVC(random_state=42))
    ])

In [43]:
param_grid = [{'vectorizer__min_df':[1, 2, 3, 4, 5],
               'vectorizer__ngram_range':[(1, 2),(1, 3)],
               'svm__C':[0.1, 0.5, 1],
               'svm__kernel':['linear', 'poly', 'rbf']
              }]
GS = GridSearchCV(PipeSVM, param_grid = param_grid, cv=5)
GS.fit(X_train, y)
GS.best_params_, GS.best_score_

({'svm__C': 1,
  'svm__kernel': 'linear',
  'vectorizer__min_df': 1,
  'vectorizer__ngram_range': (1, 3)},
 0.7835)

In [ ]:
eng = set(stopwords.words('english'))

In [290]:
PipeSVM = Pipeline([
        ('vectorizer', TfidfVectorizer(analyzer='char_wb', ngram_range=(2,8), min_df=2, stop_words='english')),
        ('svm', SVC(kernel='linear', random_state=42, C=0.8))
    ])
param_grid = {'vectorizer__sublinear_tf':[True, False]}
GS = GridSearchCV(PipeSVM, param_grid = param_grid, cv=5)
GS.fit(X_train, y)
GS.best_params_, GS.best_score_

({'vectorizer__sublinear_tf': False}, 0.7915)

In [180]:
vectorizer = TfidfVectorizer(analyzer='char_wb', min_df=2, ngram_range=(2,8), stop_words='english')
svm = SVC(kernel='linear', random_state=42, C=0.8)

X = vectorizer.fit_transform(X_train)
X_te = vectorizer.transform(X_test.text)

In [181]:
svm.fit(X, y)
svm_preds = svm.predict(X_te)

In [185]:
submission = pd.read_csv('products_sentiment_sample_submission.csv')
submission['y'] = svm_preds
submission.to_csv('SVM_predict.csv', index = None)

submission.tail()

,Id,y
495,495,0
496,496,1
497,497,0
498,498,0
499,499,1


In [187]:
from mlxtend.classifier import StackingClassifier

In [311]:
import re

In [ ]:
X_train = [' '.join([lemmer.lemmatize(word) for word in text.split(' ')])
          for text in X_train]

In [364]:
X_train = X_train.astype(str)

In [383]:
X_train[:5]

0               2  take around 10000 640x480 pictures 
1    i downloaded a trial version of computer assoc...
2    the wrt54g plus the hga7t is a perfect solutio...
3    i dont especially like how music files are uns...
4    i was using the cheapie pail  and it worked ok...
Name: Reviews, dtype: object

In [374]:
import string

In [377]:
X_train = X_train.str.replace('[{}]'.format(string.punctuation), '')

In [361]:
X_train = X_train.str.lower()

In [379]:
vectorizer = TfidfVectorizer(analyzer='char_wb', min_df=2, ngram_range=(2,8), stop_words='english')

In [370]:
vectorizer2 = TfidfVectorizer(analyzer='word', min_df=2, ngram_range=(1,2))

In [380]:
X = vectorizer.fit_transform(X_train)

In [344]:
X2 = vectorizer2.fit_transform(X_train)

In [382]:
clf1 = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
clf2 = RandomForestClassifier(random_state=42, **rf_random.best_params_)
clf3 = SVC(kernel='linear', C=0.8, random_state=42)
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=lr)

print('5-fold cross validation:\n')

for clf, label in zip([clf1, clf2, clf3, sclf], 
                      ['KNN', 
                       'Random Forest', 
                       'SVM',
                       'StackingClassifier']):

    scores = cross_val_score(clf, X, y, 
                                              cv=5, scoring='accuracy')
    print("Accuracy: {:.4f} (+/- {:.2f}) [{:s}]".format(scores.mean(), 2*scores.std(), label))

5-fold cross validation:

Accuracy: 0.7310 (+/- 0.04) [KNN]
Accuracy: 0.7655 (+/- 0.03) [Random Forest]
Accuracy: 0.7895 (+/- 0.04) [SVM]
Accuracy: 0.7655 (+/- 0.03) [StackingClassifier]


In [250]:
from nltk.stem import WordNetLemmatizer
lemmer=WordNetLemmatizer()
Xlem=[' '.join([lemmer.lemmatize(word) for word in text.split(' ')])
          for text in X_train]

In [252]:
X = vectorizer.fit_transform(Xlem)

In [ ]:
# и со стеммингом и с лемматизацией результаты хуже, такая вот история

In [216]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

In [338]:
En_stemmer = SnowballStemmer('english')

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([En_stemmer.stem(w) for w in analyzer(doc)])

In [339]:
PipeNB = Pipeline([
        ('vectorizer', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words='english')),
        ('mnb', MultinomialNB())
    ])
PipeSVM = Pipeline([
        ('vectorizer', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words='english')),
        ('svm', SVC(kernel='linear', random_state=42, C=0.5))
    ])

In [334]:
param_grid = {'vectorizer__ngram_range':[(1, 2),(1, 3)]}
GS = GridSearchCV(PipeSVM, param_grid = param_grid, cv=5)
GS.fit(reviews, y)
GS.best_params_, GS.best_score_

({'vectorizer__ngram_range': (1, 3)}, 0.7295)

In [340]:
param_grid = {'vectorizer__ngram_range':[(1, 2),(1, 3)]}
GS = GridSearchCV(PipeNB, param_grid = param_grid, cv=5)
GS.fit(reviews, y)
GS.best_params_, GS.best_score_

({'vectorizer__ngram_range': (1, 3)}, 0.7585)

In [25]:
PipeSVM = Pipeline([
        ('vectorizer', TfidfVectorizer(analyzer='char_wb', min_df=2, stop_words='english')),
        ('svm', SVC(kernel='linear', random_state=42, C=0.5))
    ])
param_grid = {'vectorizer__ngram_range':[(2,5), (3,5)]}
GS = GridSearchCV(PipeSVM, param_grid = param_grid, cv=5)
GS.fit(X_train, y)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vectorizer',
                                        TfidfVectorizer(analyzer='char_wb',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=2,
                                                        ngram_range=(1, 1),
                                 

In [26]:
GS.best_params_, GS.best_score_

({'vectorizer__ngram_range': (2, 5)}, 0.7915)

In [410]:
PipeSVM = Pipeline([
        ('vectorizer', TfidfVectorizer(analyzer='char_wb', min_df=2, ngram_range=(2,5), stop_words='english')),
        ('svm', SVC(kernel='linear', random_state=42, C=0.5))
    ])

In [412]:
PipeSVM.fit(reviews, y)
prediction_svm_charwb = PipeSVM.predict(X_test)

In [415]:
submission = pd.read_csv('products_sentiment_sample_submission.csv')
submission.head(5)

,Id,y
0,0,0
1,1,1
2,2,0
3,3,1
4,4,0


In [417]:
submission = pd.read_csv('products_sentiment_sample_submission.csv')
submission['y'] = prediction_svm_charwb
submission.to_csv('svm_charwb.csv', index = None)

submission.head()

,Id,y
0,0,1
1,1,0
2,2,1
3,3,1
4,4,0


In [ ]:
# годится, теперь немного advanced

In [264]:
import xgboost as xgb


# Hyperparameters tuning

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

# Some constants

SEED = 314159265
VALID_SIZE = 0.2
TARGET = 'outcome'

In [265]:
def intersect(l_1, l_2):
    return list(set(l_1) & set(l_2))


def get_features(train, test):
    intersecting_features = intersect(train.columns, test.columns)
    return sorted(intersecting_features)

In [299]:
# Scoring and optimization functions


def score(params):
    print("Training with params: ")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(Xtrain, label=y_train)
    dvalid = xgb.DMatrix(Xvalid, label=y_valid)
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm_model = xgb.train(params, dtrain, num_round,
                          evals=watchlist,
                          verbose_eval=True)
    predictions = gbm_model.predict(dvalid,
                                    ntree_limit=gbm_model.best_iteration + 1)
    score = acc = accuracy_score(np.argmax(y_valid,axis=1),np.argmax(gbm_model.predict(X_valid),axis=1))
    # TODO: Add the importance for the selected features
    print("\tScore {0}\n\n".format(score))
    # The score function should return the loss (1-score)
    # since the optimize function looks for the minimum
    return {'accuracy': score, 'status': STATUS_OK}


def optimize(
             #trials, 
             random_state=SEED):
    """
    This is the optimization function that given a space (space here) of 
    hyperparameters and a scoring function (score here), finds the best hyperparameters.
    """
    # To learn more about XGBoost parameters, head to this page: 
    # https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
    space = {
        'n_estimators': hp.quniform('n_estimators', 100, 1000, 1),
        'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
        # A problem with max_depth casted to float instead of int with
        # the hp.quniform method.
        'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'eval_metric': 'auc',
        'objective': 'binary:logistic',
        # Increase this number if you have more cores. Otherwise, remove it and it will default 
        # to the maxium number. 
        'nthread': 4,
        'booster': 'gbtree',
        'tree_method': 'exact',
        'silent': 1,
        'seed': random_state
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest, 
                # trials=trials, 
                max_evals=250)
    return best


In [297]:
from sklearn.model_selection import train_test_split

In [283]:
Xtrain, Xvalid, y_train, y_valid = train_test_split(X, y, test_size=VALID_SIZE,
                                random_state=SEED)

In [ ]:
best_hyperparams = optimize(
                            #trials
                            )
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)

In [133]:
def preprocess(data):
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
    def clean_special_chars(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text

    data = data.astype(str).apply(lambda x: clean_special_chars(x, punct))
    return data

In [134]:
X_train = preprocess(X_train)
X_test = preprocess(X_test)

In [13]:
import numpy as np
import pandas as pd
import os
import time
import gc
import random
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F

Using TensorFlow backend.


In [14]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [15]:
CRAWL_EMBEDDING_PATH = '/home/hq/git/crawl-300d-2M.vec'
NUM_MODELS = 1
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
MAX_LEN = 220

In [16]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [17]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def train_model(model, train, test, loss_fn, output_dim, lr=0.001,
                batch_size=512, n_epochs=4,
                enable_checkpoint_ensemble=True):
    param_lrs = [{'params': param, 'lr': lr} for param in model.parameters()]
    optimizer = torch.optim.Adam(param_lrs, lr=lr)

    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.6 ** epoch)
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    all_test_preds = []
    checkpoint_weights = [2 ** epoch for epoch in range(n_epochs)]
    
    for epoch in range(n_epochs):
        start_time = time.time()
        
        scheduler.step()
        
        model.train()
        avg_loss = 0.
        
        for data in tqdm(train_loader, disable=False):
            x_batch = data[:-1]
            y_batch = data[-1]

            y_pred = model(*x_batch)            
            loss = loss_fn(y_pred, y_batch)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            avg_loss += loss.item() / len(train_loader)
            
        model.eval()
        test_preds = np.zeros((len(test), output_dim))
    
        for i, x_batch in enumerate(test_loader):
            y_pred = sigmoid(model(*x_batch).detach().cpu().numpy())

            test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred

        all_test_preds.append(test_preds)
        elapsed_time = time.time() - start_time
        print('Epoch {}/{} \t loss={:.4f} \t time={:.2f}s'.format(
              epoch + 1, n_epochs, avg_loss, elapsed_time))

    if enable_checkpoint_ensemble:
        test_preds = np.average(all_test_preds, weights=checkpoint_weights, axis=0)    
    else:
        test_preds = all_test_preds[-1]
        
    return test_preds

In [18]:
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x
    
class NeuralNet(nn.Module):
    def __init__(self, embedding_matrix, num_aux_targets):
        super(NeuralNet, self).__init__()
        embed_size = embedding_matrix.shape[1]
        
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(0.3)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)
    
        self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        
        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)
        self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets)
        
    def forward(self, x):
        h_embedding = self.embedding(x)
        h_embedding = self.embedding_dropout(h_embedding)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        # global average pooling
        avg_pool = torch.mean(h_lstm2, 1)
        # global max pooling
        max_pool, _ = torch.max(h_lstm2, 1)
        
        h_conc = torch.cat((max_pool, avg_pool), 1)
        h_conc_linear1  = F.relu(self.linear1(h_conc))
        h_conc_linear2  = F.relu(self.linear2(h_conc))
        
        hidden = h_conc + h_conc_linear1 + h_conc_linear2
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out

In [19]:
max_features = None

In [20]:
tokenizer = text.Tokenizer(lower=False)
tokenizer.fit_on_texts(list(X_train) + list(X_test))

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(X_train, maxlen=MAX_LEN)
X_test = sequence.pad_sequences(X_test, maxlen=MAX_LEN)

In [21]:
max_features = max_features or len(tokenizer.word_index) + 1
max_features

4517

In [23]:
crawl_matrix, unknown_words_crawl = build_matrix(tokenizer.word_index, CRAWL_EMBEDDING_PATH)
print('n unknown words (crawl): ', len(unknown_words_crawl))


n unknown words (crawl):  97


In [24]:
torch.cuda.is_available()

False

In [25]:
X_train_torch = torch.tensor(X_train, dtype=torch.long)
X_test_torch = torch.tensor(X_test, dtype=torch.long)
y_train_torch = torch.tensor(y, dtype=torch.float32)

In [63]:
y_train_torch.shape

torch.Size([2000])

In [74]:
y.shape[-1]

2000

In [37]:
model.shape

AttributeError: 'NeuralNet' object has no attribute 'shape'

In [ ]:
train_dataset = data.TensorDataset(X_train_torch, y_train_torch)
test_dataset = data.TensorDataset(X_test_torch)

all_test_preds = []

for model_idx in range(NUM_MODELS):
    print('Model ', model_idx)
    seed_everything(42 + model_idx)
    
    model = NeuralNet(crawl_matrix, y.shape[-1])
    model
    
    test_preds = train_model(model, train_dataset, test_dataset, output_dim=y.shape[-1],
                             loss_fn=nn.BCEWithLogitsLoss(reduction='sum'))
    all_test_preds.append(test_preds)
    print()

Model  0
